In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchvision.models import resnet18
from torchvision.io import read_image
from torchvision import transforms

In [2]:
class Model(pl.LightningModule):
    def __init__(self):
        super(Model, self).__init__()
        self.hidden_layers = 524
        self.resnet = resnet18()
        self.in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.in_features, self.hidden_layers),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(self.hidden_layers, 196)
        )

    def forward(self, x):
        return self.resnet(x)

In [3]:
model = Model.load_from_checkpoint("../../data/model/trained_model_2021-11-12 19:41:59.859689.ckpt")

In [12]:
for i, j in model._modules['resnet'].named_children():
    print(i)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [24]:
class FeatureExtraction(nn.Module):
    def __init__(self, output_layer):
        super().__init__()
        self.output_layer = output_layer
        self.pretrained = model
        self.children_list = []
        for n, c in self.pretrained._modules['resnet'].named_children():
            print(n)
            self.children_list.append(c)
            if n == self.output_layer:
                break

        self.net = nn.Sequential(*self.children_list)
        self.pretrained = None

    def forward(self, x):
        x = self.net(x)
        return x

In [30]:
m = FeatureExtraction('avgpool')

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool


In [31]:
img = read_image(
    "/home/dauren/University/ids706 - Data Engineering/Individual projects/car-project/data/cars_train/00001.jpg")

In [32]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Lambda(lambda x: x / 255.)
])

In [33]:
img = transform(img)

In [34]:
m(img.unsqueeze(0)).shape

torch.Size([1, 512, 1, 1])